In [2]:
import pandas as pd
import numpy as np
from misc import save_to_pickle, load_from_pickle

# Load the dataset
df1 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part1.csv')
df2 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part2.csv')
df3 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part3.csv')
df4 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part4.csv')
df5 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part5.csv')
df6 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part6.csv')
df7 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part7.csv')

# Consolidating the datasets into a single data frame
df = pd.concat([df1, df2, df3, df4, df5, df6, df7])


K-fold to ensure generalisation of dataset classes, as there could potentially be some datasets that are heavy LOS / NLOS. Performing K-fold has now ensured that the dataset is evenly balanced

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df1 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part1.csv')
df2 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part2.csv')
df3 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part3.csv')
df4 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part4.csv')
df5 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part5.csv')
df6 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part6.csv')
df7 = pd.read_csv('./dataset/UWB-LOS-NLOS-Data-Set/dataset/uwb_dataset_part7.csv')

# Consolidating the datasets into a single DataFrame
df = pd.concat([df1, df2, df3, df4, df5, df6, df7])

full_training = []
full_testing = []

# Stratified sampling based on 'NLOS' column iterate through EACH df INDIVIDUALLY!
for x in range (1,8):
    train, test = train_test_split(locals()[f'df{x}'], test_size=0.2, stratify=locals()[f'df{x}']['NLOS'], random_state=42)
    full_training.append(train)
    full_testing.append(test)

# Concatenate the training and testing DataFrames
df_training = pd.concat(full_training)
df_testing = pd.concat(full_testing)

# Combine everything back together
df_combined = pd.concat([df_training, df_testing])

# Display the combined DataFrame
print(df_combined)

      NLOS  RANGE  FP_IDX  FP_AMP1  FP_AMP2  FP_AMP3  STDEV_NOISE  CIR_PWR  \
2864   1.0   5.59   746.0   5828.0   3491.0   2089.0         64.0   5332.0   
603    0.0   3.89   744.0  12137.0  18094.0  12767.0         64.0   7752.0   
3740   1.0   1.59   746.0   1832.0  10614.0  11678.0         48.0  12153.0   
5392   1.0   9.24   730.0   1008.0   1547.0   1518.0         84.0   1928.0   
5220   0.0   3.58   744.0   6502.0   6725.0   4336.0        120.0  18449.0   
...    ...    ...     ...      ...      ...      ...          ...      ...   
4973   0.0   3.74   743.0  13656.0  18752.0  10700.0         60.0   9329.0   
901    0.0   0.34   749.0  19237.0  18990.0  16568.0         84.0  12864.0   
268    1.0   5.65   744.0   6564.0   9817.0   9940.0         72.0   4496.0   
4483   1.0   5.78   750.0   4792.0  14284.0  18298.0        156.0   7599.0   
2522   1.0   1.65   749.0  10951.0   8834.0   5974.0         72.0  10648.0   

      MAX_NOISE  RXPACC  ...  CIR1006  CIR1007  CIR1008  CIR100